In [2]:
%reload_ext autoreload
%autoreload 2

import os
from pathlib import Path

print(Path().cwd())
os.chdir(Path(os.getcwd()).parents[1])
print(Path().cwd())

/home/das/QuantUS_Projs/QuantUS-Plugins-CEUS/src/processing
/home/das/QuantUS_Projs/QuantUS-Plugins-CEUS


## Select Contrast-Enhanced Ultrasound (CEUS) Cine and Parser

In [23]:
from src.image_loading.options import get_scan_loaders

print("Available scan loaders:", list(get_scan_loaders().keys()))

Available scan loaders: ['nifti']


In [ ]:
scan_type = 'nifti'

scan_path = '/media/das/TOSHIBA EXT/Animal Data/Molecular/m907/20150410121501.008 KDR.nii.gz'
scan_loader_kwargs = {}

In [ ]:
from src.entrypoints import scan_loading_step

image_data = scan_loading_step(scan_type, scan_path, **scan_loader_kwargs)

## Load Segmentation

Assumes same segmentation for each frame

In [24]:
from src.seg_loading.options import get_seg_loaders

print("Available segmentation loaders:", list(get_seg_loaders().keys()))

Available segmentation loaders: ['nifti_voi', 'load_bolus_mask']


In [ ]:
seg_type = 'nifti_voi'

seg_path = '/media/das/TOSHIBA EXT/Animal Data/Molecular/m907/nifti_segmentation_QUANTUS/20150410121501.008_KDR_segmentation.nii.gz'
seg_loader_kwargs = {}

In [ ]:
from src.entrypoints import seg_loading_step

seg_data = seg_loading_step(seg_type, image_data, seg_path, scan_path, **seg_loader_kwargs)

## CEUS Quantitative Temporal Curve Analysis

In [5]:
from src.ttc_analysis.options import get_analysis_types, get_required_kwargs

all_analysis_types, all_analysis_funcs = get_analysis_types()
print("Available analysis types:", list(all_analysis_types.keys()))

Available analysis types: ['ttc_curves']


In [6]:
analysis_type = 'ttc_curves'

print("Available analysis functions:", list(all_analysis_funcs[analysis_type].keys()))

Available analysis functions: ['pyradiomics']


In [8]:
analysis_funcs = ['pyradiomics']

# Find all required kwargs for the analysis functions
analysis_funcs = analysis_funcs if len(analysis_funcs) else list(all_analysis_funcs[analysis_type].keys())
required_kwargs = get_required_kwargs(analysis_type, analysis_funcs)
print("Required kwargs for current analysis:", required_kwargs)

Required kwargs for current analysis: ['pyradiomics_config_path']


In [9]:
analysis_kwargs = {
    'pyradiomics_config_paths': [
        '/home/das/QuantUS_Projs/QuantUS-Plugins-CEUS/configs/pyradiomics/moderate.yml',
    ],
    'curves_output_path': 'test.csv',
}

In [ ]:
from src.entrypoints import analysis_step

analysis_obj = analysis_step(analysis_type, image_data, seg_data, analysis_funcs, **analysis_kwargs)

Computing curves: 100%|██████████| 19/19 [00:30<00:00,  1.60s/it]
